## **Load libraries and data**

In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install emoji

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [ ]:
import pandas as pd
import string
import emoji
import re
import pythainlp
from nltk.tokenize import RegexpTokenizer
from pythainlp.corpus.common import thai_words
from pythainlp import Tokenizer, word_tokenize
from pythainlp.util import normalize
from pythainlp import correct
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [ ]:
pythainlp.__version__

'2.3.1'

In [ ]:
df = pd.read_csv('CustomerReviews.csv')

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN


## **Tokenize Words**

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
punctuations = list(string.punctuation)
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ', '!!!!', '++', '..', './', '::', '^^','xx']
"""'\u200b',':','-' """

screening_words = stopwords + removed_words +punctuations

"""
words = set(thai_words())  
words.add("อสิมอฟ") 
custom_tokenizer = Tokenizer(words)
print("custom dictionary :", custom_tokenizer.word_tokenize(text))
"""

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)



def tokenize_with_space(sentence):
  sentence = re.sub(r'\d+', '', sentence)
  sentence = deEmojify(sentence)
  merged = ''
  words = word_tokenize(normalize(str(sentence)), engine="multi_cut", keep_whitespace=False)
  for word in words:
    #print(word)
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,เคยลอง,สาขายู,นม,อลล์,รอบนี้,สาขา,เ..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"ร้านบุฟเฟ่ต์,โปรโมชั่น,หัว,บาท,น้ำ,VAT,ทานวัน,..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จักร้าน,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,จาก..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,มากค่ะ,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"ก็ตามที่,จ่าย,ราคา,บาท,จ่าย,เงินสดนะ,ราคานี้,น..."


In [ ]:

for index, row in df.iterrows():
    print(" index {} \n Review {} \n".format(index+1,row['Review']))



 index 1 
 Review ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักสุดๆสะอาดสะอ้านใส่ใจค่อนข้างประทับใจกับการบริการรสชาติอาหารอันนี้แปลก​ใหม่
เซ็ท560 บาท
อาหารจำกัด​ไม่ได้​หลากหลาย​แต่เพียง​พอกับความต้องการ
เราไม่กินเนื้อ​ กินสันคอ4 ถาด​ เป็ด1
แฟนกินเนื้อ​ คุณ​ภาพ​ดี
น้ำซุป​ดำ​กับเผ็ด
ที่สำคัญคือน้ำจิ้มอร่อยมากไม่เคยกินรสชาติแบบนี้มาก่อนเป็นเต้าเจี้ยวนิดๆแต่ก็ไม่ได้เคลมก็รสชาติดีนะถามราคาโดยรวมก็ถือว่าคุ้มเพราะว่าโมจิอร่อยมากไอติมก็อร่อยไม่ต้องออกไปกินข้างนอกสรุปรวมๆละกันว่าประทับใจในบริการและคุณภาพของอาหาร 

 index 2 
 Review นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไดซ์ คุณภาพเค้ามาตราฐานจริงๆ วันนี้มีโอกาสมาทานก่อนร้านปิด 30 นาที ทางร้านก็บริการดี จะมีก็แต่อาหาร เครื่องดื่มที่ทางร้านเริ่มทะยอยเก็บ แต่เราก็เข้าใจเพราะทางร้านแจ้งรายละเอียดการงดรับออร์เดอร์แล้ว 
           สาขานี้ พนักงานบริการดี น่ารักเป็นกันเอง ขออะไรก็ได้หมด ไม่มีเหวี่ยง เคยเจอสาขา พระราม 9 คือ ไม่โอเค พนักงาน ไม่พร้อมทำงานมาก CTW แม้จะเป็นพรีเมียมการบริการก็สู้สาขานี้ไม่ได้ Terminal 21 ลูกค้าเยอะ 

 index 3 
 Revie

In [ ]:

for index, row in df.iterrows():
    print(" index {} \n Review {} \n".format(index+1,row['Review_tokenized']))

 index 1 
 Review บริการ,ดีมาก,พนักงาน,น่ารัก,สะอาดสะอ้าน,ใส่ใจ,ประทับใจ,การบริการรสชาติ,อาหารอันนี้,แปลกใหม่,เซ็ท,บาท,อาหาร,จำกัด,หลากหลาย,แต่เพียงพอกับ,ความต้องการ,กิน,เนื้อ,กิน,สัน,คอ,ถาด,เป็ด,แฟน,กิน,เนื้อ,คุณภาพดี,น้ำ,ซุป,ดำ,เผ็ด,น้ำจิ้ม,อร่อย,เคยกิน,รสชาติ,แบบนี้,มาก่อน,เต้าเจี้ยว,เคลม,รสชาติดีนะ,ถามราคา,โดยรวม,คุ้ม,โมจิ,อร่อย,ไอติมก็,อร่อย,ไม่ต้องออก,กิน,ข้างนอก,สรุป,ประทับใจ,บริการ,คุณภาพ,ของอาหาร 

 index 2 
 Review นึกถึง,ชาบู,ญี่ปุ่น,คิดถึง,โม,โม่,พาราไดซ์,คุณภาพ,เค้า,มาตรา,ฐาน,มีโอกาสมาทานก่อนร้าน,นาที,ร้านก็,บริการ,ดี,อาหาร,เครื่องดื่ม,ที่ทาง,ทะยอย,แจ้ง,รายละเอียด,การงดรับ,ออร์เดอร์,สาขานี้,พนักงานบริการดี,น่ารัก,เป็นกันเอง,เหวี่ยง,เจอ,สาขา,พระราม,โอเค,พนักงาน,ทำงานมาก,CTW,พรีเมียม,การบริการก็,สู้,สาขานี้,Terminal,ลูกค้า 

 index 3 
 Review ทาน,นั่ง,โต๊ะ,หม้อ,เดียวกันได้,โต๊ะ,ยังมี,ฉาก,กั้น,น้ำ,ซุป,เลือก,รส,น้ำจิ้ม,แตกต่างกัน,ตามน้ำ,ซุป,เลือก,ชาบู,น้ำ,ดำ,น้ำ,ซุป,เห็ด,ชาบู,น้ำ,ดำ,หวาน,เค็ม,รสชาติ,เข้มข้น,เสิร์ฟ,ไข่,สด,เป็นผู้ใหญ่,ชอบ,หวาน,ชอบ,น้ำ,ซุป,เห็ด,มีชื่อ,ภาษา,ญี่ปุ่

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(df['Review_tokenized'])

In [ ]:
cvec.vocabulary_

{'CTW': 0,
 'G': 1,
 'HALL': 2,
 'MCC': 3,
 'Menu': 4,
 'Narai': 5,
 'Net': 6,
 'Number': 7,
 'Shabushi': 8,
 'Terminal': 9,
 'VAT': 10,
 'mall': 11,
 'net': 12,
 'pizza': 13,
 'pizzeria': 14,
 'union': 15,
 'ก': 16,
 'กก': 17,
 'กรรมวิธี': 18,
 'กรอบดี': 19,
 'กลยุทธ์': 20,
 'กลับมา': 21,
 'กลายเป็นมา': 22,
 'กั้น': 23,
 'กา': 24,
 'การงดรับ': 25,
 'การบริการก็': 26,
 'การบริการก็ดี': 27,
 'การบริการรสชาติ': 28,
 'การรอ': 29,
 'กาแฟ': 30,
 'กำ': 31,
 'กิน': 32,
 'กี่': 33,
 'กุ้ง': 34,
 'กๆ': 35,
 'ก็ตามที่': 36,
 'ก์': 37,
 'ขนมอร่อย': 38,
 'ขนาด': 39,
 'ขวัญใจ': 40,
 'ของการกิน': 41,
 'ของกิน': 42,
 'ของคาว': 43,
 'ของบน': 44,
 'ของมา': 45,
 'ของสดก็': 46,
 'ของหวาน': 47,
 'ของหวานที่': 48,
 'ของหวานราคา': 49,
 'ของหวานอร่อย': 50,
 'ของอาหาร': 51,
 'ขอมารีวิวร้าน': 52,
 'ขาดสารอาหาร': 53,
 'ขาบูชิต': 54,
 'ขึ้นราคา': 55,
 'ขึ้นใจ': 56,
 'ข่าว': 57,
 'ข้น': 58,
 'ข้อดี': 59,
 'ข้างนอก': 60,
 'ข้าว': 61,
 'ข้าวทานกับ': 62,
 'ข้าวปั้น': 63,
 'ข้าวมันไก่': 64,
 'คน': 65,
 'คนละ': 66,
 '

## **Create Dictionary**

In [ ]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['การบริการรสชาติ', 'กิน', 'ของอาหาร', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุณภาพดี', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดีมาก', 'ถาด', 'ถามราคา', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'มาก่อน', 'รสชาติ', 'รสชาติดีนะ', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อาหาร', 'อาหารอันนี้', 'เคยกิน', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แต่เพียงพอกับ', 'แบบนี้', 'แปลกใหม่', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้องออก', 'ไอติมก็', 'CTW', 'Terminal', 'การงดรับ', 'การบริการก็', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ดี', 'ทะยอย', 'ทำงานมาก', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการดี', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอกาสมาทานก่อนร้าน', 'รายละเอียด', 'ร้านก็', 'ลูกค้า', 'สาขา', 'สาขานี้', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [ ]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 392 ms, sys: 6.23 ms, total: 399 ms
Wall time: 399 ms


In [ ]:
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.060654 -0.050995       1        1  24.780177
5     -0.089764 -0.001767       2        1  21.423223
6      0.039326  0.066188       3        1  20.520725
4     -0.011192 -0.022973       4        1  12.955331
3      0.006763  0.009688       5        1   8.820389
2     -0.005953 -0.001454       6        1   7.345617
1      0.000167  0.001314       7        1   4.154538, topic_info=      Term       Freq      Total Category  logprob  loglift
56      ดี  11.000000  11.000000  Default  30.0000  30.0000
29   อร่อย  16.000000  16.000000  Default  29.0000  29.0000
36   เนื้อ  18.000000  18.000000  Default  28.0000  28.0000
276   กุ้ง   8.000000   8.000000  Default  27.0000  27.0000
113    ผัก   6.000000   6.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
222     คน   0.309534  10.467931   Topic7  -5.1471  -0.3400
94     ชอบ   0.298222   8.477933   Topic7  -5.1843  -0.1664
10     ซุป   0.308088  13.884996   Topic7  -5.1518  -0.6272
122   ราคา   0.287125   9.352277   Topic7  -5.2223  -0.3025
71    สาขา   0.256646   7.056677   Topic7  -5.3345  -0.1331

[450 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
331       1  0.856500            net
332       2  0.566302         กลับมา
501       1  0.856780  การบริการก็ดี
1         1  0.287537            กิน
1         2  0.287537            กิน
...     ...       ...            ...
166       2  0.326009          ไอติม
166       3  0.163005          ไอติม
166       4  0.163005          ไอติม
166       6  0.163005          ไอติม
330       2  0.923909             ้ะ

[390 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 7, 5, 4, 3, 2])

In [ ]:
model.show_topic(3)

[('เนื้อ', 0.020571558),
 ('ชาบู', 0.014418333),
 ('กิน', 0.013972978),
 ('เลือก', 0.013778678),
 ('อร่อย', 0.013217764),
 ('สั่ง', 0.012268368),
 ('ชอบ', 0.010255045),
 ('อาหาร', 0.0091974335),
 ('น้ำ', 0.00820262),
 ('บริการ', 0.0079623535)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df[['Restaurant', 'Review', 'topics']]

,Restaurant,Review,topics
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,0
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,4
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,6
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,3
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,3
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,2
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,0
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,6
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,4
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,5
